In [ ]:
%pip install -r requirements.txt

In [ ]:
# nohup python train_data_gen.py --audio_dir "/data/AAG/MTech_Project_Data/speech_data_filter"  --cnt 2500  --train_dir "train_data" --test_dir "test_data" --output_path "pkl_data"  --target_fs_values 16000 --clipping_thresholds 0.1 0.2 --time_clip 1 --win_len 500  --win_shift 125 --delta 300 --s_ratio 0.9 > data_gen_log.txt 2>&1 &

In [ ]:
# !python segregate.py --audio_dir "/data2/AAG/Ausio_Declip/MTech_Project_Data/speech_data_filter" \
#     --cnt 10 --train_dir "train_data" --test_dir "test_data" --s_ratio 0.9 

In [ ]:
# nohup python train_data_gen.py --audio_dir "/data2/AAG/Audio_Declip/train_data" --output_path "pkl_data" --target_fs_values 16000 --clipping_thresholds 0.2 0.4 0.6 0.8 --time_clip 1 --win_len 500 --win_shift 125 --delta 300 > train_gen.log 2>&1 &
    

In [ ]:
# nohup python training.py --pkl_path pkl_data/training_data.pkl --epochs 500 --batch_size 128 --save_path saved_models --plot_path loss_plots --checkpoint_freq 50 --val_split 0.2 --resume --val > training_log.log 2>&1 &

In [ ]:
# import pickle

# # Define the file path on the server
# file_path = '/data2/AAG/Audio_Declip/pkl_data/training_data.pkl'

# # Load the .pkl file
# with open(file_path, 'rb') as file:
#     data = pickle.load(file)

# # Extract a small portion of the data (e.g., first 1000 rows if it's a DataFrame or a subset if it's a different structure)
# small_data = data[:10000] if isinstance(data, list) else data

# # Save the small data as a new .pkl file
# small_data_path = '/data2/AAG/Audio_Declip/pkl_data/small_training_data.pkl'
# with open(small_data_path, 'wb') as file:
#     pickle.dump(small_data, file)

# print(f"Small data saved to {small_data_path}")

Small data saved to /data2/AAG/Audio_Declip/pkl_data/small_training_data.pkl


In [8]:
!python training.py --pkl_path pkl_data/small_training_data.pkl \
    --epochs 50 \
    --batch_size 512 \
    --save_path saved_models \
    --plot_path loss_plots \
    --checkpoint_freq 5 \
    --val \
    --val_split 0.2 \
    --resume 

Training samples: 8000, Validation samples: 2000
Using device: cuda
Epoch [1/50], Train Loss: 1.689895, Val Loss: 1.979414, Train DFT Loss: 1.584059, Val DFT Loss: 1.889423, Train Sparsity Loss: 0.211672, Val Sparsity Loss: 0.179981
Epoch [2/50], Train Loss: 1.114992, Val Loss: 1.281029, Train DFT Loss: 1.057002, Val DFT Loss: 1.239929, Train Sparsity Loss: 0.115979, Val Sparsity Loss: 0.082199
Epoch [3/50], Train Loss: 0.827978, Val Loss: 0.759687, Train DFT Loss: 0.795634, Val DFT Loss: 0.726308, Train Sparsity Loss: 0.064689, Val Sparsity Loss: 0.066758
Epoch [4/50], Train Loss: 0.605242, Val Loss: 0.511260, Train DFT Loss: 0.581141, Val DFT Loss: 0.474899, Train Sparsity Loss: 0.048202, Val Sparsity Loss: 0.072723
Epoch [5/50], Train Loss: 0.466938, Val Loss: 0.418420, Train DFT Loss: 0.457206, Val DFT Loss: 0.407950, Train Sparsity Loss: 0.019464, Val Sparsity Loss: 0.020939
Checkpoint saved to saved_models/checkpoint.pt
Epoch [6/50], Train Loss: 0.427069, Val Loss: 0.438720, Trai

In [3]:
!python evaluate.py --model_path "saved_models/final/complex_dft_unet_final.pth" \
    --test_audio_dir "custom_sound" \
    --output_dir "custom_sound_output" \
    --target_fs_values 16000 \
    --clipping_thresholds 0.2 \
    --time_clip 1 \
    --factor 0.2 \
    --eval_mode 0 \
    --dynamic 0 \
    --save 0 \
    --delta 300 \
    --c_win 500 \
    --r_mode 0

Processing files:   0%|                                   | 0/1 [00:00<?, ?it/s]
Processing: 1221-135767-0012.wav (fs=16000, threshold=0.2, duration=1)
Generating clipped signal...
Processing files: 100%|███████████████████████████| 1/1 [00:06<00:00,  6.43s/it]


In [ ]:
# !python plot_loss_history.py --history_file saved_models/loss_history.json --output_path final_plots

In [2]:
import pandas as pd

# File paths
baseline_file = "/data2/AAG/Audio_Declip/custom_output/evaluation_results_baseline_model.xlsx"
dynamic_file = "/data2/AAG/Audio_Declip/custom_output/evaluation_results_dynamic_model.xlsx"
ml_file = "/data2/AAG/Audio_Declip/custom_output/evaluation_results_ml_model.xlsx"

# Load data
df_baseline = pd.read_excel(baseline_file)
df_dynamic = pd.read_excel(dynamic_file)
df_ml = pd.read_excel(ml_file)

# Combine in a dict
models = {
    "Baseline": df_baseline,
    "Dynamic": df_dynamic,
    "ML": df_ml
}

# Metrics to summarize
metrics = ['delta_sdr', 'delta_pesq', 'processing_time', 'cycles']
thresholds = [0.1, 0.3]

# Print header
print(f"{'Threshold':<10} {'Model':<10} {'Delta SDR':<20} {'Delta PESQ':<20} {'Time (s)':<20} {'Cycles':<20}")

# Loop by threshold first
for threshold in thresholds:
    for model_name, df in models.items():
        group = df[df['threshold'] == threshold]
        values = []
        for metric in metrics:
            mean = group[metric].mean()
            std = group[metric].std()
            values.append(f"{mean:.2f} ({std:.2f})")
        print(f"{threshold:<10} {model_name:<10} {values[0]:<20} {values[1]:<20} {values[2]:<20} {values[3]:<20}")


Threshold  Model      Delta SDR            Delta PESQ           Time (s)             Cycles              
0.1        Baseline   18.33 (2.17)         1.01 (0.42)          4.76 (1.04)          36733.32 (8044.73)  
0.1        Dynamic    18.29 (2.10)         0.97 (0.43)          4.37 (0.92)          26530.55 (5652.93)  
0.1        ML         17.96 (2.20)         1.05 (0.40)          4.73 (0.75)          20777.75 (4455.39)  
0.3        Baseline   20.69 (4.07)         1.49 (0.55)          5.71 (1.02)          45176.12 (8067.86)  
0.3        Dynamic    20.66 (4.02)         1.48 (0.55)          5.27 (0.93)          32789.71 (5745.81)  
0.3        ML         21.03 (3.85)         1.53 (0.53)          5.32 (0.72)          25391.88 (4556.79)  


In [1]:
import pickle

# Path to your .pkl file
file_path = "pkl_data/training_data.pkl"

# Load the pickle file
with open(file_path, "rb") as f:
    data = pickle.load(f)

In [4]:
!python plot_loss_history.py --history_file /data2/AAG/Audio_Declip/saved_models/loss_history.json

Plots saved to loss_plots
